## 用requests模拟登陆豆瓣

### 导入需要的库

In [1]:
# -*- encoding:utf-8 -*-  
#import urllib2  
#import urllib  
import re  
import cookielib  
import requests  
from PIL import Image
import cStringIO

### 定义登录url 表单信息 HTTP头

In [ ]:
loginUrl = 'https://www.douban.com/accounts/login'  

formData={    
    "form_email":'你的账号',            #这里填写你的账号
    "form_password":'你的密码' ,        #这里填写你的密码
    'source':'index_nav'
}

headers={  
    'Accept'    : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',   
    'Host' : 'www.douban.com',  
    'Referer' : 'https://www.douban.com/',  
    'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; rv:48.0)'  
} 

### 获取session数据

In [ ]:
session = requests.session()
req = session.get('https://www.douban.com/',headers=headers, allow_redirects=False)

### 模拟登陆
#### 针对验证码问题，通过正则表达式获取验证码url，并手动填入

In [ ]:
#正则提取ID
pattern = re.compile('<img id="captcha_image".*?id=(.*?)&',re.S)     
result = re.search(pattern,req.text)
id = result.group(1)

#正则提取验证码链接
pattern = re.compile('<img id="captcha_image".*?src="(.*?) alt="captcha',re.S)     
result = re.search(pattern,req.text)
imageURL = result.group(1).strip()

#打开图像链接，输出图片
request=urllib2.Request(imageURL,headers=headers)
respHtml = urllib2.urlopen(request).read()
img = Image.open(cStringIO.StringIO(respHtml))  
print u'请输入你看到的字母'
img.show()

#人眼识别，填入验证码信息
checkCode = raw_input()
#表单中加入captcha-solution
formData["captcha-solution"]=checkCode
#表单加入id信息
formData["captcha-id"]=id
#发送post请求
session.post(loginUrl,data=formData,headers=headers) 
#站内的测试链接，用来判断是否登入成功
url = "https://www.douban.com/people/90868630/"
#用session访问
code = session.get(url, headers=headers, allow_redirects=False)
if code.status_code==200 :
    print u'登陆成功'
else:
print u'登录失败'